In [1]:
import pandas as pd
import cobra

from cobra.io import load_model

import numpy as np
import pickle

In [2]:
model = cobra.io.read_sbml_model('models\\Recon3D.xml')

In [3]:
model.genes[:3]

[<Gene 0 at 0x1eef3de10d0>,
 <Gene 26_AT1 at 0x1eef3dd28d0>,
 <Gene 314_AT1 at 0x1ee9f266410>]

In [4]:
model.metabolites[:3]

[<Metabolite 10fthf_c at 0x1eef3ea2850>,
 <Metabolite 10fthf_l at 0x1eef3ea2810>,
 <Metabolite 10fthf_m at 0x1eef3ea3050>]

In [5]:
model.reactions[:3]

[<Reaction 24_25DHVITD3tm at 0x1eea06c6b90>,
 <Reaction 25HVITD3t at 0x1eef3d40a50>,
 <Reaction COAtl at 0x1eea06c7210>]

In [ ]:
model.

In [6]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
thmtp_c,DM_thmtp_c,1000,12,4.09%
12ppd__R_e,EX_12ppd__R_e,250.8,3,0.26%
5aop_e,EX_5aop_e,1000,5,1.70%
HC00250_e,EX_HC00250_e,1000,0,0.00%
HC00900_e,EX_HC00900_e,1000,4,1.36%
HC01361_e,EX_HC01361_e,1000,9,3.07%
Lkynr_e,EX_Lkynr_e,105.1,10,0.36%
acetone_e,EX_acetone_e,427.7,3,0.44%
alltn_e,EX_alltn_e,1000,4,1.36%
atp_e,EX_atp_e,1000,10,3.41%


In [ ]:
rx = model.reactions.get_by_id("2AMACHYD")

In [ ]:
rx.gene_reaction_rule

In [ ]:
for r in model.reactions[:100]:
    gpr = r.gene_reaction_rule
    if gpr:
        print(gpr)

In [ ]:
model.reactions.BIOMASS_maintenance.build_reaction_string()